## Import libraries

In [92]:
import numpy as np
import open3d as o3d
from scipy.optimize import least_squares
from sklearn.cluster import DBSCAN

## Load point cloud

In [93]:
pcd = o3d.io.read_point_cloud("filtered2.pcd")

## Visualize point cloud

In [94]:
o3d.visualization.draw_geometries([pcd])

## Voxelize point cloud

In [95]:
downpcd = pcd.voxel_down_sample(voxel_size=0.005)
o3d.visualization.draw_geometries([downpcd])
points = np.asarray(downpcd.points)

## RANSAC Plane

In [20]:
def fit_plane(points):
    centroid = np.mean(points, axis=0)
    _, _, vh = np.linalg.svd(points - centroid)
    normal = vh[2, :]
    d = -np.dot(normal, centroid)
    return np.append(normal, d)

def ransac_plane(points, threshold, iterations):
    best_inliers = []
    best_params = None

    for _ in range(iterations):
        sample = points[np.random.choice(points.shape[0], 3, replace=False)]
        params = fit_plane(sample)
        inliers = []

        for point in points:
            distance = np.abs(np.dot(params[:3], point) + params[3]) / np.linalg.norm(params[:3])
            if distance < threshold:
                inliers.append(point)

        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            best_params = params

    return best_params, np.array(best_inliers)

## RANSAC Sphere

In [7]:
def fit_sphere(points):
    def residuals(params, points):
        x0, y0, z0, r = params
        return np.sqrt((points[:, 0] - x0)**2 + (points[:, 1] - y0)**2 + (points[:, 2] - z0)**2) - r

    # Initial guess for the sphere parameters
    x0, y0, z0 = np.mean(points, axis=0)
    r0 = np.mean(np.linalg.norm(points - [x0, y0, z0], axis=1))
    initial_guess = [x0, y0, z0, r0]

    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x

def ransac_sphere(points, threshold, iterations):
    best_inliers = []
    best_params = None

    for _ in range(iterations):
        sample = points[np.random.choice(points.shape[0], 4, replace=False)]
        params = fit_sphere(sample)
        inliers = []

        for point in points:
            distance = np.abs(np.sqrt((point[0] - params[0])**2 + (point[1] - params[1])**2 + (point[2] - params[2])**2) - params[3])
            if distance < threshold:
                inliers.append(point)

        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            best_params = params

    return best_params, np.array(best_inliers)

## RANSAC Cone

In [13]:
def fit_cone(points):
    def residuals(params, points):
        x0, y0, z0, a, b, c, theta = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  # Normalize the direction vector
        
        # Vector from the cone vertex to the point
        vector_to_point = points - np.array([x0, y0, z0])
        
        # Projection of the vector onto the cone's axis
        projection_length = np.dot(vector_to_point, direction_vector)
        projection = projection_length[:, np.newaxis] * direction_vector
        
        # Perpendicular distance from the point to the axis of the cone
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        
        # Expected distance based on the cone's angle
        expected_distance = np.tan(theta) * projection_length
        
        # Residuals are the difference between the actual and expected distances
        return perpendicular_distance - expected_distance

    # Initial guess for the cone parameters
    x0, y0, z0 = np.mean(points, axis=0)
    direction = np.array([0, 0, 1])  # Initial guess: cone points upwards
    theta0 = np.pi / 6  # Initial guess: 30 degrees opening angle
    initial_guess = [x0, y0, z0, direction[0], direction[1], direction[2], theta0]

    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x

def ransac_cone(points, threshold, iterations):
    best_inliers = []
    best_params = None

    for _ in range(iterations):
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        params = fit_cone(sample)
        inliers = []

        for point in points:
            x0, y0, z0, a, b, c, theta = params
            direction_vector = np.array([a, b, c])
            direction_vector /= np.linalg.norm(direction_vector)
            
            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length
            
            distance = np.abs(perpendicular_distance - expected_distance)
            if distance < threshold:
                inliers.append(point)

        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            best_params = params

    return best_params, np.array(best_inliers)

## RANSAC Cylinder

In [42]:
def fit_cylinder(points):
    def residuals(params, points):
        x0, y0, z0, a, b, c, r = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  # Normalize the direction vector
        
        # Vector from the cylinder's axis point to the point
        vector_to_point = points - np.array([x0, y0, z0])
        
        # Projection of the vector onto the cylinder's axis
        projection_length = np.dot(vector_to_point, direction_vector)
        projection = projection_length[:, np.newaxis] * direction_vector
        
        # Perpendicular distance from the point to the axis of the cylinder
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        
        # Residuals are the difference between the actual perpendicular distance and the cylinder's radius
        return perpendicular_distance - r

    # Initial guess for the cylinder parameters
    x0, y0, z0 = np.mean(points, axis=0)
    direction = np.array([0, 0, 1])  # Initial guess: cylinder axis pointing upwards
    r0 = np.mean(np.linalg.norm(points - np.array([x0, y0, z0]), axis=1))  # Approximate initial radius
    initial_guess = [x0, y0, z0, direction[0], direction[1], direction[2], r0]

    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x

def ransac_cylinder(points, threshold, iterations):
    best_cost = 0.0
    best_params = None
    best_inliers = []

    for _ in range(iterations):
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        params = fit_cylinder(sample)
        
        x0, y0, z0, a, b, c, r = params
        cylinder_axis_point = np.array([x0, y0, z0])
        cylinder_axis_direction = np.array([a, b, c])
        
        # Calculate cost for the current model
        cost = cylinder_score_function(points, cylinder_axis_point, cylinder_axis_direction, r)
        
        # Check if this model is better
        if cost > best_cost:
            best_cost = cost
            best_params = params
            
            # Recompute inliers for the best model
            inliers = []
            for point in points:
                vector_to_point = point - cylinder_axis_point
                projection_length = np.dot(vector_to_point, cylinder_axis_direction)
                projection = projection_length * cylinder_axis_direction
                perpendicular_distance = np.linalg.norm(vector_to_point - projection)
                
                distance = np.abs(perpendicular_distance - r)
                if distance < threshold:
                    inliers.append(point)
            
            best_inliers = inliers

    return best_params, np.array(best_inliers)

## Cost function

In [62]:
def cylinder_score_function_edge_weighted(point_cloud, cylinder_axis_point, cylinder_axis_direction, cylinder_radius, tolerance=0.1):
    score = 0
    axis_direction_normalized = cylinder_axis_direction / np.linalg.norm(cylinder_axis_direction)
    max_distance = 0
    
    for p in point_cloud:
        # Vector from axis point to current point
        v = p - cylinder_axis_point
        
        # Project vector onto the cylinder axis direction to find the closest point on the axis
        projection_length = np.dot(v, axis_direction_normalized)
        closest_point_on_axis = cylinder_axis_point + projection_length * axis_direction_normalized
        
        # Radial distance from point to the cylinder's axis
        radial_distance = np.linalg.norm(p - closest_point_on_axis)
        
        # Update maximum distance to prioritize boundary points
        max_distance = max(max_distance, radial_distance)
        
        # Calculate the distance from the point to the cylinder's surface
        distance_to_surface = np.abs(radial_distance - cylinder_radius)
        
        # Adjust score contribution based on how close the point is to the cylinder surface
        if distance_to_surface < tolerance:
            score += 1 / (distance_to_surface + 1e-6)
        else:
            score -= distance_to_surface**2
    
    # Weight the score based on how close the model is to the boundary of the point cloud
    score *= max_distance / (cylinder_radius + 1e-6)
    
    return score

In [67]:
from scipy.optimize import least_squares
from scipy.spatial import ConvexHull

def fit_cylinder(points):
    def residuals(params, points):
        x0, y0, z0, a, b, c, r = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  # Normalize the direction vector
        
        # Vector from the cylinder's axis point to the point
        vector_to_point = points - np.array([x0, y0, z0])
        
        # Projection of the vector onto the cylinder's axis
        projection_length = np.dot(vector_to_point, direction_vector)
        projection = projection_length[:, np.newaxis] * direction_vector
        
        # Perpendicular distance from the point to the axis of the cylinder
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        
        # Residuals are the difference between the actual perpendicular distance and the cylinder's radius
        return perpendicular_distance - r

    # Initial guess for the cylinder parameters
    x0, y0, z0 = np.mean(points, axis=0)
    direction = np.array([0, 0, 1])  # Initial guess: cylinder axis pointing upwards
    r0 = np.mean(np.linalg.norm(points - np.array([x0, y0, z0]), axis=1))  # Approximate initial radius
    initial_guess = [x0, y0, z0, direction[0], direction[1], direction[2], r0]

    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x

def find_boundary_points(points):
    hull = ConvexHull(points)
    return points[hull.vertices]

def cylinder_score_function_with_regularization(point_cloud, cylinder_axis_point, cylinder_axis_direction, cylinder_radius, tolerance=0.1, min_radius=1.0):
    score = 0
    axis_direction_normalized = cylinder_axis_direction / np.linalg.norm(cylinder_axis_direction)
    max_distance = 0
    
    for p in point_cloud:
        v = p - cylinder_axis_point
        projection_length = np.dot(v, axis_direction_normalized)
        closest_point_on_axis = cylinder_axis_point + projection_length * axis_direction_normalized
        radial_distance = np.linalg.norm(p - closest_point_on_axis)
        max_distance = max(max_distance, radial_distance)
        distance_to_surface = np.abs(radial_distance - cylinder_radius)
        
        if distance_to_surface < tolerance:
            score += 1 / (distance_to_surface + 1e-6)
        else:
            score -= distance_to_surface**2
    
    score *= max_distance / (cylinder_radius + 1e-6)
    
    # Add a penalty if the cylinder radius is smaller than expected
    if cylinder_radius < min_radius:
        score -= 1000 * (min_radius - cylinder_radius)
    
    return score

def ransac_cylinder_with_boundary(points, threshold, iterations):
    best_score = -float('inf')
    best_params = None
    best_inliers = []

    for _ in range(iterations):
        # Use boundary points to drive the fit
        boundary_points = find_boundary_points(points)
        params = fit_cylinder(boundary_points)
        
        x0, y0, z0, a, b, c, r = params
        cylinder_axis_point = np.array([x0, y0, z0])
        cylinder_axis_direction = np.array([a, b, c])
        
        # Calculate score for the current model
        score = cylinder_score_function_with_regularization(points, cylinder_axis_point, cylinder_axis_direction, r)
        
        # Check if this model has a higher score
        if score > best_score:
            best_score = score
            best_params = params
            
            # Recompute inliers for the best model
            inliers = []
            for point in points:
                vector_to_point = point - cylinder_axis_point
                projection_length = np.dot(vector_to_point, cylinder_axis_direction)
                projection = projection_length * cylinder_axis_direction
                perpendicular_distance = np.linalg.norm(vector_to_point - projection)
                
                distance = np.abs(perpendicular_distance - r)
                if distance < threshold:
                    inliers.append(point)
            
            best_inliers = inliers

    return best_params, np.array(best_inliers)

In [96]:
import numpy as np
from scipy.optimize import least_squares
from scipy.spatial import ConvexHull, KDTree
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.csgraph import connected_components

def calculate_normals(points, k=10):
    """
    Estimate normals for each point in the point cloud using PCA on the k-nearest neighbors.
    """
    tree = KDTree(points)
    normals = []
    for point in points:
        _, idx = tree.query([point], k=k)
        neighbors = points[idx[0]]
        pca = np.linalg.svd(neighbors - np.mean(neighbors, axis=0))[2]
        normals.append(pca[-1])
    return np.array(normals)

def connectivity_measure(points, inlier_mask, k=10):
    """
    Compute the largest connected component of inliers using a k-nearest neighbor graph.
    """
    # Ensure that inlier_mask is a boolean array and has the correct shape
    inlier_mask = np.asarray(inlier_mask, dtype=bool)
    
    if len(inlier_mask) != len(points):
        raise ValueError("inlier_mask must be the same length as points.")
    
    # Extract the points that are inliers
    inlier_points = points[inlier_mask]
    n_inliers = len(inlier_points)
    
    if n_inliers == 0:
        return 0
    
    # Ensure k is not greater than the number of inliers
    k = min(k, n_inliers)
    
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(inlier_points)
    adjacency_matrix = nbrs.kneighbors_graph(inlier_points).toarray()
    n_components, labels = connected_components(adjacency_matrix)
    
    largest_component_size = max(np.bincount(labels))
    return largest_component_size

def score_function(points, normals, cylinder_axis_point, cylinder_axis_direction, cylinder_radius, epsilon, alpha, k=10):
    """
    Score a cylinder candidate based on support, normal alignment, and connectivity.
    """
    axis_direction_normalized = cylinder_axis_direction / np.linalg.norm(cylinder_axis_direction)
    inlier_mask = np.zeros(len(points), dtype=bool)  # Initialize inlier_mask as a boolean array

    # Compute the inliers within the epsilon band
    for i, point in enumerate(points):
        v = point - cylinder_axis_point
        projection_length = np.dot(v, axis_direction_normalized)
        closest_point_on_axis = cylinder_axis_point + projection_length * axis_direction_normalized
        radial_distance = np.linalg.norm(point - closest_point_on_axis)
        
        if np.abs(radial_distance - cylinder_radius) <= epsilon:
            inlier_mask[i] = True

    # Filter by normal alignment
    for i in range(len(points)):
        if inlier_mask[i]:
            normal_deviation = np.arccos(np.clip(np.dot(normals[i], axis_direction_normalized), -1.0, 1.0))
            if normal_deviation > alpha:
                inlier_mask[i] = False

    # Compute connectivity score
    connectivity_score = connectivity_measure(points, inlier_mask, k)

    return connectivity_score

def ransac_cylinder_with_advanced_scoring(points, threshold, iterations, epsilon, alpha):
    best_score = -float('inf')
    best_params = None
    best_inliers = []

    # Estimate normals for the entire point cloud
    normals = calculate_normals(points)

    for _ in range(iterations):
        # Randomly sample points to fit the cylinder
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        params = fit_cylinder(sample)
        
        x0, y0, z0, a, b, c, r = params
        cylinder_axis_point = np.array([x0, y0, z0])
        cylinder_axis_direction = np.array([a, b, c])
        
        # Calculate the score using the advanced scoring function
        score = score_function(points, normals, cylinder_axis_point, cylinder_axis_direction, r, epsilon, alpha)
        
        # Check if this model has a higher score
        if score > best_score:
            best_score = score
            best_params = params
            
            # Recompute inliers for the best model
            best_inliers = []
            for i, point in enumerate(points):
                # Here, we'll simply check if the point is within the epsilon band and has the correct normal alignment
                inlier = score_function([point], [normals[i]], cylinder_axis_point, cylinder_axis_direction, r, epsilon, alpha)
                if inlier > 0:
                    best_inliers.append(point)

    return best_params, np.array(best_inliers)

## Nomrals

In [99]:
normals = o3d.geometry.estimate_normals(points, knn = 30)

AttributeError: module 'open3d.cpu.pybind.geometry' has no attribute 'estimate_normals'

## Run RANSAC

In [97]:
cylinder_params, cylinder_inliers = ransac_cylinder_with_advanced_scoring(points, threshold=0.1, iterations=500, epsilon=0.1, alpha=0.087)
#sphere_params, sphere_inliers = ransac_sphere(points, threshold=0.005, iterations=500)
#cone_params, cone_inliers = ransac_cone(points, threshold=0.005, iterations=500)

TypeError: only integer scalar arrays can be converted to a scalar index

## Visualize results

In [76]:
#plane_cloud = o3d.geometry.PointCloud()
#plane_cloud.points = o3d.utility.Vector3dVector(plane_inliers)
#plane_cloud.paint_uniform_color([0,1,0])

#sphere_cloud = o3d.geometry.PointCloud()
#sphere_cloud.points = o3d.utility.Vector3dVector(sphere_inliers)
#sphere_cloud.paint_uniform_color([0,0,1])

#cone_cloud = o3d.geometry.PointCloud()
#cone_cloud.points = o3d.utility.Vector3dVector(cone_inliers)
#cone_cloud.paint_uniform_color([1,0,0])

cylinder_cloud = o3d.geometry.PointCloud()
cylinder_cloud.points = o3d.utility.Vector3dVector(cylinder_inliers)
cylinder_cloud.paint_uniform_color([1,0,0])

downpcd.paint_uniform_color([0,0,0])

o3d.visualization.draw_geometries([downpcd, cylinder_cloud])

RuntimeError: 

In [29]:
cylinder_cloud = o3d.geometry.PointCloud()
cylinder_cloud.points = o3d.utility.Vector3dVector(cylinder_inliers)
cylinder_cloud.paint_uniform_color([0,1,0])

PointCloud with 829 points.

In [30]:
o3d.visualization.draw_geometries([downpcd, cylinder_cloud])